# Tiny Islands
Below is the basic game logic for 'Tiny Islands'. Our objective is to code an interactable and updateable game state adhering to the rules of 'Tiny Island' that we can later use to for both the visualization of the game interface as well as the application of reinforcement learning models.

The game state should keep track of:
1. The number of points currently represented on the board
2. The number of moves already played / whether or not the game has finished
3. The position of unoccupied tiles on the board 

The game state should take in:
1. The type of terrain supplied
2. The X and Y coordinates of the tile

## Board Rules

The board is consisted of tiles, each of which can be occupied with at most one terrain. Each terrain may interact with each other (same or different depending on unique attributes) on the board, generating and/or deducting the total number of points.

### (Bonus Rules) These types of terrain are, respectively:

1. BOATS: **1pt** for each square seperating them from the nearest **BOAT** or **ISLAND**
   
2. WAVES: **2pt** for each **WAVE** that isn't the same row or column as another **WAVE**
   
3. BEACH: **1pt** for each **BEACH** that is touching(4) an island, but not on the island
   
4. HOUSES: **1pt** for each unique terrain that is nearby(8) but not a **HOUSE**
   
5. CHURCHES: 
   - **2pt** for each **HOUSE** nearby
   - **1pt** for each **HOUSE** on the same island
   - **0pt** all together if there's 2 or more **CHURCHES** on the same **ISLAND**
  
6. FOREST: **2pt** for each **FOREST** in a group (touching(4)) in total, minus **2pt** (once for each group)
   
7. MOUNTAIN: **2pt** for each **FOREST** nearby(8)

### (Island Rules) But hold up! What are **ISLANDS**?

- The game has a total of 30 turns, After **Turn 30**, the board evaluation is the final score
- On each 10th turn (**Turn 10** **Turn 20** & **Turn 30**) instead of inserting terrain as listed above, the player must "draw" an **ISLAND**
- **ISLANDS** are drawn between tiles, on their borders/sides(4), Each tile border has **Length 1**
- **ISLANDS** must be enclosed. All enclosed tiles are part of an **ISLAND**
- The player is given a maximum of **Length 24** to draw an **ISLAND**

*RECURSIVE ISLAND: It is possible for an **ISLAND** to have a 'hole' in it. In the case where one **ISLAND** encloses another, the inner '**ISLAND**' is not considered an **ISLAND**.

### (Penalty Rules) The introduction of **ISLANDS** bring more rules to the game:

1. BOATS: **-5pt** and **INACTIVE** for each **BOAT** that's also part of an **ISLAND** 
   
2. WAVES: 
   - **-5pt** and **INACTIVE** for each **WAVE** that's also part of an **ISLAND**
 - [**VERY IMPORTANT**] **0pt** for each **WAVE** that's touching(4) an **ISLAND**

3. BEACH: 
   - **-5pt** and **INACTIVE** for each **BEACH** that's also part of an **ISLAND**
   - [**VERY IMPORTANT**] **0pt** for each **BEACH** that's not touching(4) an **ISLAND**
   
4. HOUSES: **-5pt** and **INACTIVE** for each **HOUSE** that's not part of an **ISLAND**
   
5. CHURCHES: **-5pt** and **INACTIVE** for each **CHURCH** that's not part of an **ISLAND**
  
6. FOREST: **-5pt** and **INACTIVE** for each **FOREST** that's not part of an **ISLAND**
   
7. MOUNTAIN: **-5pt** and **INACTIVE** for each **MOUNTAIN** that's not part of an **ISLAND**

### Evaluation Order:

1. Check and Set Inactive
* Offshore Beaches though **ACTIVE**, do not interact with other terrain. It is important to keep its **ACTIVE** status since:
  - All Inactive terrain are penalized
  - All Inactive terrain are dimmed on display 
2. Evaluate Terrain based on Board Interactions 1-7
3. Check and Deduct Penalties

## Objects and Data Structures

With the board rules all laid out, we then move onto designing and structuring the game code. This is a crucial step, since training machine learning models are especially computationally intensive. Other than simplicity and speed, I also worry about whether reinforcement learning can be effectively applied here. 

At this early stage in the project I probably shouldn't think too far ahead, but I'd like list my concerns here for future reference. When considering traditional projects, I find the most plausible objective to be deciding the optimal position for any terrain at every step in the game. However, when we play the game we are choosing from 2 random terrain types each assigned to a random column/row on the board. In this case, it might be useful to generate a value map for each step and then comparing the highest value for each choice.

Q: Should we bind the "1 out of 2 terrain+row/column" choice with the choice of the specific tile later on?
A: Yes, since we're only trying to optimize the final score at the very end.

A major issue is how to combine the entirely different "island drawing" step into this traditional model. From a macro point of view, "island drawing" is basically choosing from a list of tiles that fit the "Length 24 or smaller" requirement. We can train the model based on this variable, but how would it reflect its interaction with "terrain fitting"? Terrain such as "Boats" and "Beach" demand strict "island planning" for it to take effect. Through experience, "Boats" are a wild card that can sometimes rack up huge amounts of points while "Beach" is a "versa-tile" (see what i did there) that can easily slide into any "build". Training the models seperately will overlook all of these strategies. Moreover, there are only 3 instances of island drawing in an entire game, meaning that model training might not be very effective, leave alone optimal. 

Reflecting upon these thoughts, here's a list of points of concern for future reference:
- Must check if turn options are randomly generately with uniform distribution. If yes, we can omit the "double choice split" mentioned in the above Q&A.
- Must find a way to incorporate the two turn types into one model

With all that said, lets get to code design. For the board, we need tiles. Tiles should have a X and Y coordinate to represent position, which is useful for determining possible insertion tiles and evaluating tile interactions. It must also store both the terrain type and ideally if its on an island, next to one, or offshore. Both must be integers for computational reasons. We also need to store the overall steps and points in the board object (as integers) for recording purposes.

Related functions involve inserting new tiles, returning unoccupied tiles and evaluating the board to update the score/points system.

In [5]:
# type:                 loc:                st:
# 0: Unoccupied         -1: Shore           0: Inactive
# 1: Boasts             0: Ocean            1: Active
# 2: Waves              1: Island 1
# 3: Beach              2: Island 2
# 4: Houses             3: Island 3
# 5: Churches
# 6: Forest
# 7: Mountains

class Tile:
    def __init__(self, type, loc, st):
        self.type = type
        self.loc = loc
        self.st = st

In [6]:
class Board:
    def __init__(self):
        # Initialize Score
        self.score = 0
        # Initialize Tiles List
        self.tiles = []
        # Initialize Tiles Values
        for y in range(0,9):
            row = []
            for x in range(0,9):
                row.append(Tile(0,0,0))
            self.tiles.append(row)
    
    # Set Floor of 0 for Total Score
    def scrflr(self):
        if self.score < 0:
            self.score = 0
    
    # Return List of Tuple XY for Unoccupied Tiles
    def unoc(self):
        # Initialize List
        unoc = []
        # Search For Unoccupied Tiles
        rownum = 0
        for row in self.tiles:
            colnum = 0
            for tile in row:
                if tile.type == 0:
                    unoc.append((colnum, rownum))
                colnum += 1
            rownum += 1
        return unoc
    
    # Insert New Tile
    def insr(self, x, y, type):
        # Locate Tile Position
        tile = self.tiles[y][x]
        # Update Tile Type
        tile.type = type
        # Activate Terrain Status, Await Evaluation
        tile.st = 1
    
    # Evaluate board
    def eval(self):
        for row in self.tiles:
            for tile in row:
                if tile.st == 1:
                    self.pen(tile)
        self.bonus()
    
    # Evaluate Penalty
    def pen(self, t):
        if t.type in range(1,4):
            if t.loc in range(1,4):
                t.st = 0
                self.score -= 5
        elif t.type in range(4,8):
            if t.loc not in range(1,4):
                t.st = 0
                self.score -= 5
        self.scrflr()
    
    def bonus(self):
        shps = []
        isls = []
        wvs = []
        hses = []
        chrhs = []
        grps = []
        frst = []
        mtns = []
        rownum = 0

        for row in self.tiles:
            colnum = 0
            for tile in row:
                if tile.type == 1 and tile.st == 1:
                    shps.append((colnum,rownum))
                if tile.loc in [1,2,3]:
                    isls.append((colnum,rownum))
                if tile.type == 2 and tile.st == 1 and tile.loc == 0:
                    wvs.append((colnum,rownum))
                if tile.type == 3 and tile.st == 1 and tile.loc == -1:
                    self.score += 1
                if tile.type == 4 and tile.st == 1:
                    hses.append((colnum,rownum))
                if tile.type == 5 and tile.st == 1:
                    chrhs.append((colnum,rownum))
                if tile.type == 6 and tile.st == 1:
                    frst.append((colnum,rownum))
                if tile.type == 7 and tile.st == 1:
                    mtns.append(tile)
                colnum += 1
            rownum += 1
        
        for s in shps:
            targs = shps + isls
            targs.remove(s)
            if targs:
                dlist = []
                for tar in targs:
                    dist = abs(tar[0] - s[0]) + abs(tar[1] - s[1])
                    dlist.append(dist)
                self.score += min(dlist)
        
        for w in wvs:
            targs = wvs
            targs.remove(w)
            bonus = 2
            for tar in targs:
                if w[0] == tar[0] or w[1] == tar[1]:
                    bonus = 0
                    break
            self.score += bonus
        
        for h in hses:
            unique = []
            for ytry in range(-1,2):
                for xtry in range(-1,2):
                    if xtry != 0 and ytry != 0:
                        xnear = h[0] + xtry
                        ynear = h[1] + ytry
                        if xnear <= 8 and ynear <= 8:
                            targ = self.tiles[ynear][xnear]
                            if targ.type not in (0,4) and targ.type not in unique:
                                unique.append(targ.type)
            self.score += len(unique)

        for c in chrhs:
            crusade = False
            inf = chrhs
            inf.remove(c)
            # Check for Crusade
            for i in inf:
                if c.loc == i.loc:
                    crusade = True
            if not crusade:
                nrow = 0
                for row in self.tiles:
                    ncol = 0
                    for targ in row:
                        if targ.type == 4 and c.loc == targ.loc:
                            self.score += 1
                            for ytry in range(-1,2):
                                for xtry in range(-1,2):
                                    if xtry != 0 and ytry != 0:
                                        xnear = c[0] + xtry
                                        ynear = c[1] + ytry
                                        if targ[0] == xnear and targ[1] == ynear:
                                            self.score += 1
                        ncol += 1
                    nrow += 1
        
        for f in frst:
            for i in (-1, 1):
                for grp in grps:
                    for targ in grp:
                        if (f[0] + i == targ[0] or f[0]== targ[0]) and (f[1] == targ[1] or f[1] + i == targ[1]):
                            grp.append(f)
                            registered = True
                            break
                if registered == True:
                    break
            if not registered:
                newgrp = []
                newgrp.append(f)
                grps.append(newgrp)
        for grp in grps:
            self.score += len(grp) * 2 - 2

        for m in mtns:
            for ytry in range(-1,2):
                for xtry in range(-1,2):
                    if xtry != 0 and ytry != 0:
                        xnear = m[0] + xtry
                        ynear = m[1] + ytry
                        if xnear <= 8 and ynear <= 8:
                            targ = self.tiles[ynear][xnear]
                            if targ[0] == m[0] + xtry and targ[1] == m[1] + ytry:
                                if targ.type == 5:
                                    self.score += 2

In [13]:
from tkinter import *

main_window = Tk()
b = Board()
# Labels
rownum = 0
for row in b.tiles:
    colnum = 0
    for tile in row:
        Label(main_window, text=0).grid(row=rownum, column=colnum)
        colnum += 1
    rownum += 1

Label(main_window, text=0).grid(row=rownum, column=colnum)

main_window.mainloop()

### Testing

In [9]:
b = Board()
print(b.score)
b.insr(7,7,1)
b.eval()
print(b.score)
b.insr(0,0,1)
b.eval()
print(b.score)

0
0
28
